# PlayHQ Fixture Scraping

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ssardina/tapp-fixture/blob/main/playhq_scrape.ipynb)

This system allows to scrape game fixtures from [PlayHQ](http://playhq.com/
) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs). 

It will produce a CSV file ready to be uploaded as Schedule in [TeamApp](https://brunswickmagicbasketball.teamapp.com/).

The *Public* APIs only require a header parameters to get a successful response, which includes `x-api-key` (also referred to as the Client ID) and `x-phq-tenant` (refers to the sport/association - in this case `bv`). Note that the Private APIs are not available to clubs and associations.

Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

Contact: Sebastian Sardina (sssardina@gmail.com)

In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re
import os
import calendar, datetime

# Set-up everything if running in Google Colab
if "COLAB_GPU" in os.environ:
  %pip install pyshorteners
  %pip install coloredlogs
  for f in ['utils.py', 'playhq.py', 'config.py']:
    if not os.path.exists(f):
      !wget "https://raw.githubusercontent.com/ssardina/tapp-fixture/main/{f}"

from config import *
import utils
import playhq as phq

## 1. Set-up application

First, creating a connection to the PlayHQ Public API. Remember to set-up file `config.py` for club and season configuration.


The *Public* APIs only require the below header parameters to get a successful response:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com. This key can be stored in a file `x_api_key.txt` or it will be asked interactively by the notebook otherwise. In many cases, the feature to create new API credentials is disabled for a user and can only be actioned by a Super Administrator role within the Play HQ portal.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.

The game date (`GAME_DATE`) is assumed to be the upcoming Saturday, but a specific day can be set here by uncommenting and editing the second line.

In [ ]:
GAME_DATE = utils.next_day(calendar.SATURDAY) # get the date of the upcoming Saturday (game day in competition)
# GAME_DATE = datetime.date(2022, 8, 27)  # can also fix a particular game day
GAME_DATE_TIMESTAMP = pd.to_datetime(GAME_DATE).tz_localize(TIMEZONE)
GAME_DATE_NAME = GAME_DATE_TIMESTAMP.strftime("%A %B %d, %Y (%Y/%m/%d)") # Saturday August 06, 2022

# if no x-api-key is defined, ask the user for it
if X_API_KEY is None:
  X_API_KEY = input("Enter your x-api-key:")
print("x-api-key is now defined!")

phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE)
print(f"Set-up games for club {CLUB_NAME} for upcoming Saturday is: {GAME_DATE_NAME}")

PLAYHQ_URL=f"https://bv.playhq.com/org/{ORG_ID}/games?date={GAME_DATE_TIMESTAMP.strftime('%Y-%m-%d')}"
print("Game day at PlayHQ:", PLAYHQ_URL)
print("Club PlayHQ link: ", PLAYHQ_CLUB_SEASON)

Now get the teams of the club; it will be used below.

In [ ]:
season_id = phq_club.get_season_id(SEASON)
teams_df = phq_club.get_season_teams(season_id)
teams_df

## 2. Get the upcoming games for those teams

Using the teams of the club, extract all upcoming games for the Club's teams.

In [ ]:
upcoming_games_df = phq_club.get_games(teams_df, GAME_DATE_TIMESTAMP)

if upcoming_games_df is not None:
    print(f'There were {upcoming_games_df.shape[0]} games extracted for game day: {GAME_DATE_NAME}')
    upcoming_games_df[phq.GAMES_COLS]
else:
    print("No games for date: ", GAME_DATE_NAME)

## 3. Convert to TeamApp CSV format

Next, we convert the PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

In [ ]:
games_tapps_df = utils.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP, game_duration=45)
print("Done computing the games for Teams App")

games_tapps_df.sample(3)

Inspect how the description of one of the games will look like:

In [ ]:
# Inspect description of one record
print("Description for:", games_tapps_df.iloc[18]['event_name'], "\n")
print(games_tapps_df.iloc[18]['description'])

### Extract BYE games

We now extract the teams for which we couldn't scrape a game. In most cases this means a BYE for those teams.

In [ ]:
# Extract the date of the round
# date = team_apps_csv_df.iloc[1]['start_date']
print(f"Extract BYE games for games on {GAME_DATE_NAME}")

playing_teams = upcoming_games_df['team_id'].tolist()
bye_teams = teams_df.loc[~teams_df['id'].isin(playing_teams)]['name'].tolist()
bye_teams = list(map(lambda x: re.search("U.*", x).group(0), bye_teams))

if bye_teams:
    games_bye_df = utils.build_teamsapp_bye_schedule(bye_teams, GAME_DATE)
    print(f"Bye teams ({len(bye_teams)}): ", bye_teams)
else:
    print("No BYE games this round...")

Finally, put together upcoming games and BYE games in a single table that will later be used to produce a CSV for TeamApp schedule import.

In [ ]:
if bye_teams:
    team_apps_csv_df = pd.concat([games_tapps_df, games_bye_df])
    team_apps_csv_df.drop_duplicates(inplace=True)
    team_apps_csv_df.reset_index(inplace=True, drop=True)
else:
    team_apps_csv_df = games_tapps_df

team_apps_csv_df.sample(4)

## 5. Save to CSV file for Teams App import

In this section, we will produce the CSV file to be imported into TeamAPP as well as pickle files saving the computed dataframes.

We start by reporting the games to be written into Schedule CSV file and **CHECKING THAT ALL IS GOOD TO GO!**

Particularly, look for games that are schedule but **PENDING** and without all details (time or venue).

In [28]:
team_apps_csv_df.columns
team_apps_csv_df[['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]
# team_apps_csv_df

,team_name,opponent,start_date,start_time,venue,court
0,U8 Mixed Purple,Rebels U8 Mixed Red,2022-10-22,08:30:00,Coburg Basketball Stadium,Court 4
1,U8 Mixed Gold,SUBC-CM81,2022-10-22,08:30:00,Oak Park Stadium,Court 1
2,U8 Mixed Black,Melbourne Warriors U8 Mixed,2022-10-22,08:30:00,Coburg Basketball Stadium,Court 1
3,U18 Girls Gold,SUBC-CG182,2022-10-22,15:15:00,Oak Park Stadium,Court 1
4,U18 Boys Purple,Piranhas U18 Boys Black,2022-10-22,16:00:00,St John's College (Preston),Court 1
5,U16 Girls Purple,Newlands U16 Girls WILDCATS,2022-10-22,13:45:00,Coburg Basketball Stadium,Court 1
6,U16 Girls Gold,STARS U16 Girls TB,2022-10-22,13:45:00,Dallas Brooks Community Primary School,Court 1
7,U16 Boys Purple,Jets U16 Boys Red,2022-10-22,14:30:00,Coburg Senior High School,Court 1
8,U16 Boys Gold,St Fidelis U16 Boys Gold,2022-10-22,14:30:00,Dallas Brooks Community Primary School,Court 1
9,U16 Boys Diamond,Jets U16 Boys Orange,2022-10-22,13:00:00,Northcote High School,Court 1


We stop the execution here if we are running all Jupyter notebook.

In [ ]:
raise SystemExit("Stop right there! Continue below to produce the CSV file if needed.")

### 5.2. Check changes with previous saves

If the schedule was generated before, check if the new one differs with the one saved already.

First, let us define the files that we will save to disk.

In [ ]:
import datetime
import os
import shutil

now = datetime.datetime.now() # current date and time
now_str = now.strftime("%Y-%m-%d_%H:%M:%S")
game_date_str = GAME_DATE.strftime('%Y_%m_%d')

file_csv = os.path.join(OUTPUT_PATH, f"schedule-teamsapp-{game_date_str}.csv")
file_upcoming_pkl = os.path.join(OUTPUT_PATH, f"upcoming_games_df-{GAME_DATE.strftime('%Y_%m_%d')}.pkl")
file_team_apps_csv = os.path.join(OUTPUT_PATH, f"team_apps_csv_df-{GAME_DATE.strftime('%Y_%m_%d')}.pkl")

Next, let's check if there was a saved file for the upcoming game day.

In [ ]:
cols = ['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']

changed_games_df = None
if os.path.exists(file_team_apps_csv):
    print("There was already a schedule saved, recovering it to compare...")
    old_team_apps_csv_df = pd.read_pickle(file_team_apps_csv)

    teams_changed = pd.concat([team_apps_csv_df[cols], old_team_apps_csv_df[cols]]).drop_duplicates(keep=False)['team_name'].unique()
    print("Teams whose games have changed (updated, new, dropped):", teams_changed)

    old_games_df = old_team_apps_csv_df[cols].query("team_name in @teams_changed")
    new_games_df = team_apps_csv_df[cols].query("team_name in @teams_changed")
    changed_games_df = new_games_df.merge(old_games_df, how="inner", on="team_name", suffixes=('_new', '_old'))

# Show changes if any...
(changed_games_df is not None) and changed_games_df

### 5.3. Write a TeamAPP Schedule CSV & Datafarmes Pickles

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [ ]:
import datetime
import os
import shutil

now = datetime.datetime.now() # current date and time
now_str = now.strftime("%Y-%m-%d_%H:%M:%S")
game_date_str = GAME_DATE.strftime('%Y_%m_%d')

if not os.path.exists(OUTPUT_PATH):
  os.makedirs(OUTPUT_PATH)

print('Saving TeamAPP schedule CSV file and Dataframes for games:', GAME_DATE_NAME)

file_csv = os.path.join(OUTPUT_PATH, f"schedule-teamsapp-{game_date_str}.csv")
file_upcoming_pkl = os.path.join(OUTPUT_PATH, f"upcoming_games_df-{GAME_DATE.strftime('%Y_%m_%d')}.pkl")
file_team_apps_csv = os.path.join(OUTPUT_PATH, f"team_apps_csv_df-{GAME_DATE.strftime('%Y_%m_%d')}.pkl")

for f in [file_csv, file_upcoming_pkl, file_team_apps_csv]:
  if os.path.exists(f):
    print("Backup file", f)
    shutil.copy(f, f + ".bak")

print('File to save TeamApp schedule:', file_csv)
team_apps_csv_df.to_csv(file_csv, index=False)

print('Saving dataframe pickle:', file_upcoming_pkl)
upcoming_games_df.to_pickle(file_upcoming_pkl)
print('Saving dataframe pickle:', file_team_apps_csv)
team_apps_csv_df.to_pickle(file_team_apps_csv)

print(f"Finished saving csv and data-frame files: {now.strftime('%d/%m/%Y, %H:%M:%S')}")

# ------------ END FIXTURE PUBLISHING ------------

### Check a particular team

In [29]:
upcoming_games_df.query("team_name == 'U12 Boys Diamond'")

,id,team_name,team_id,status,url,createdAt,updatedAt,pool,competitors,grade_id,...,venue_surfaceName,venue_surfaceAbbreviation,venue_address_line1,venue_address_postcode,venue_address_suburb,venue_address_state,venue_address_country,venue_address_latitude,venue_address_longitude,schedule_timestamp
21,4f33617d-d2e0-43ee-b356-bd69acd20bce,U12 Boys Diamond,7931f0c7-a524-43ff-89a3-a329d7e12690,UPCOMING,https://www.playhq.com/basketball-victoria/org...,2022-10-17 22:17:05+11:00,2022-10-19 12:43:02+11:00,None,[{'id': '530229de-2221-4f04-9183-b608082298de'...,cae7adb9-98b6-41fa-9590-48cf9ec53f22,...,Court 1,Crt1,180 OHEA ST,3058,COBURG,VIC,Australia,-37.7351,144.95113,2022-10-22 13:00:00+11:00


In [39]:
team = "U10 Girls Gold"

print(games_tapps_df.query("team_name == @team")['description'].values[0])
team_apps_csv_df.query("team_name == @team")[['team_name', 'opponent', 'start_date', 'start_time', 'venue', 'court']]


RSVP mandatory for the game.

Opponent: Warriors U10 Girls
Venue: Oak Park Stadium (Court 1)
Address: 9 Hillcrest Road, Oak Park 
Google Maps coord: https://maps.google.com/?q=(-37.713213,144.925617)

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- No food in the venue and pickup your rubbish.
- Games will have 2x20 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: https://tinyurl.com/2kcur65a
Check the round in PlayHQ: https://tinyurl.com/2pwusro8
All clubs in PlayHQ: https://bit.ly/bmbc-s22



,team_name,opponent,start_date,start_time,venue,court
25,U10 Girls Gold,Warriors U10 Girls,2022-10-22,09:15:00,Oak Park Stadium,Court 1
